In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt 
import skvideo
skvideo.setFFmpegPath('D:\\Taishin\\ffmpeg-20200115-0dc0837-win64-static\\bin')
import skvideo.io
import dlib

D:\Anaconda\lib\site-packages\skvideo\__init__.py:306: UserWarning: ffmpeg/ffprobe not found in path: D:\實習\ffmpeg-20200115-0dc0837-win64-static\bin
  warnings.warn("ffmpeg/ffprobe not found in path: " + str(path), UserWarning)


In [22]:
newpath = 'D:\\Anaconda\\Library\\etc\\haarcascades\\'

face_cascade = cv2.CascadeClassifier(newpath+'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(newpath+'haarcascade_eye.xml')

In [23]:
def detect(img, idx):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 
                                          scaleFactor=1.3, 
                                          minNeighbors=5,
                                          minSize=(220,220),)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,
                                            scaleFactor=1.05,
                                            minNeighbors=5,
                                            minSize=(50,50),)
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),5)
        for (ex,ey,ew,eh) in eyes:
            img = cv2.rectangle(img,(x+ex,y+ey),(x+ex+ew,y+ey+eh),(0,255,0),5)
    print('Working with %s frame' % idx)
    return img

In [24]:
videogen = skvideo.io.vreader('test1.mp4')

In [ ]:
plt.ion()
for idx,frame in enumerate(videogen):
    frame = detect(frame, idx)
    plt.imshow(frame)
    plt.pause(.01)
    plt.cla()  # Clear axis
    plt.clf()  # Clear figure

In [58]:
plt.ioff()

# dlib

In [12]:
from imutils.face_utils import FaceAligner
from imutils.video import count_frames
import imutils
import numpy as np
import dlib
import cv2
import os
import glob
from skimage.measure import compare_ssim
import time

In [13]:
def writeObjects(label, bbox):
    with open(object_xml_file) as file:
        file_content = file.read()

    file_updated = file_content.replace("{NAME}", label)
    file_updated = file_updated.replace("{XMIN}", str(bbox.left()))
    file_updated = file_updated.replace("{YMIN}", str(bbox.bottom()))
    file_updated = file_updated.replace("{XMAX}", str(bbox.right()))
    file_updated = file_updated.replace("{YMAX}", str(bbox.top()))

    return file_updated

In [14]:
# 取得人臉位置資訊並儲存成xml格式
def generateXML(img, filename, fullpath, bboxes):
    
    xmlObject = ""
    labelName = "face"
    
    for bbox in bboxes:

        xmlObject = xmlObject + writeObjects(labelName, bbox)

#     print("SHAPE:", img.shape)

    with open(xml_file) as file:

        xmlfile = file.read()

    h,w = img.shape[0],img.shape[1]

    xmlfile = xmlfile.replace( "{WIDTH}", str(w) )

    xmlfile = xmlfile.replace( "{HEIGHT}", str(h) )

    xmlfile = xmlfile.replace( "{FILENAME}", str(filename) )

#     xmlfile = xmlfile.replace( "{PATH}", fullpath + filename )

    xmlfile = xmlfile.replace( "{PATH}", fullpath +'{0}.jpg'.format(filename) )
    
    xmlfile = xmlfile.replace( "{OBJECTS}", xmlObject )

    return xmlfile

#xmlContent = generateXML(來源圖片, xml檔名, 完整的xml檔案path, 該張圖片Dlib所偵測到的所有face boxes)

# xmlContent = generateXML(frame, xmlfilename, datasetPath + labelPath + xmlfilename, faces)

In [15]:
# 人臉辨識模型
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=160)

face_filename = 1
def detect(img, idx, total):
    # 影像灰階化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 臉部辨識
    faces = detector(gray, 1)
    # 把圖片中每張臉存下來時使用
    for face in faces:
        faceAligned = fa.align(gray, gray, face)
        global face_filename # 可將變數儲存於function外
        #cv2.imwrite('D:\\Taishin\\face3_160_try\\{0}.png'.format(face_filename), faceAligned)
        cv2.imwrite('D:\\Taishin\\yolo\\images_test\\{0}.jpg'.format(face_filename), img) #.png
        face_filename += 1
        
    if face_filename % 100 == 0 :
        print('Working with {0} frame. completed {1:.2f} %'.format(idx, idx/float(total)*100))


xml_file = "xml_file.txt"
object_xml_file = "xml_object.txt"
detect_video = 'test2.mp4' # 目標影片
videoCapture = cv2.VideoCapture(detect_video)
success, frame = videoCapture.read()
frame_counter = 0
# videoCapture.set(cv2.CAP_PROP_FPS,10.0)
fps = videoCapture.get(cv2.CAP_PROP_FPS) # 一秒取得幾幀
timestamps  = [videoCapture.get(cv2.CAP_PROP_POS_MSEC)]
frame_total = count_frames(detect_video) # 計算總幀數
timegap = 500
while success:
    if videoCapture.get(cv2.CAP_PROP_POS_MSEC) >= timegap*frame_counter :
        
        detect(frame, frame_counter, frame_total)
#         xmlContent = generateXML(frame, face_filename,'D:\\Taishin\\yolo\\lables_test\\', faces)
#         file = open('D:\\Taishin\\yolo\\lables_test\\{0}.xml'.format(face_filename), "w")
#         file.write(xmlContent)
#         file.close

        timestamps.append(videoCapture.get(cv2.CAP_PROP_POS_MSEC))
        frame_counter += 1
    success, frame = videoCapture.read()
        


print("Done!!")

Working with 102 frame. completed 3.11 %
Working with 190 frame. completed 5.79 %
Done!!


In [43]:
frame.shape[0]

360

In [11]:
frame.shape[0]

AttributeError: 'NoneType' object has no attribute 'shape'

In [43]:
frame.shape[0]

360

360

In [16]:
timestamps

[0.0,
 0.0,
 41.708333333333336,
 83.41666666666667,
 125.12499999999999,
 166.83333333333334,
 208.54166666666666,
 250.24999999999997,
 291.9583333333333,
 333.6666666666667,
 375.375,
 417.0833333333333,
 458.79166666666663,
 500.49999999999994,
 542.2083333333333,
 583.9166666666666,
 625.625,
 667.3333333333334,
 709.0416666666666,
 750.75,
 792.4583333333333,
 834.1666666666666,
 875.875,
 917.5833333333333,
 959.2916666666666,
 1000.9999999999999,
 1042.7083333333333,
 1084.4166666666665,
 1126.125,
 1167.8333333333333,
 1209.5416666666667,
 1251.25,
 1292.9583333333333,
 1334.6666666666667,
 1376.375,
 1418.0833333333333,
 1459.7916666666665,
 1501.5,
 1543.2083333333333,
 1584.9166666666665,
 1626.625,
 1668.3333333333333,
 1710.0416666666667,
 1751.75,
 1793.4583333333335,
 1835.1666666666665,
 1876.8749999999998,
 1918.5833333333333,
 1960.2916666666665,
 2001.9999999999998,
 2043.7083333333335,
 2085.4166666666665,
 2127.125,
 2168.833333333333,
 2210.5416666666665,
 2252.2

In [18]:
s, ms = divmod(1236472051807, 1000) #(商數,餘數)

In [19]:
'{}.{:03d}'.format(time.strftime('%H:%M:%S', time.gmtime(s)), int(ms))

'00:27:31.807'

In [22]:
timestamps

[0.0,
 0.0,
 41.708333333333336,
 83.41666666666667,
 125.12499999999999,
 166.83333333333334,
 208.54166666666666,
 250.24999999999997,
 291.9583333333333,
 333.6666666666667,
 375.375,
 417.0833333333333,
 458.79166666666663,
 500.49999999999994,
 542.2083333333333,
 583.9166666666666,
 625.625,
 667.3333333333334,
 709.0416666666666,
 750.75,
 792.4583333333333,
 834.1666666666666,
 875.875,
 917.5833333333333,
 959.2916666666666,
 1000.9999999999999,
 1042.7083333333333,
 1084.4166666666665,
 1126.125,
 1167.8333333333333,
 1209.5416666666667,
 1251.25,
 1292.9583333333333,
 1334.6666666666667,
 1376.375,
 1418.0833333333333,
 1459.7916666666665,
 1501.5,
 1543.2083333333333,
 1584.9166666666665,
 1626.625,
 1668.3333333333333,
 1710.0416666666667,
 1751.75,
 1793.4583333333335,
 1835.1666666666665,
 1876.8749999999998,
 1918.5833333333333,
 1960.2916666666665,
 2001.9999999999998,
 2043.7083333333335,
 2085.4166666666665,
 2127.125,
 2168.833333333333,
 2210.5416666666665,
 2252.2

In [23]:
def get_time(timestamps) : 
    time_info = []
    seconds_temp = []
    mileseconds = []
    for ms in timestamps : 
        seconds_temp.append(divmod(ms,1000.0)[0])
        mileseconds.append(divmod(ms,1000.0)[1])
       
    time_info.append(mileseconds)
    minutes_temp = []
    seconds = []
    for s in seconds_temp : 
        minutes_temp.append(divmod(s,60.0)[0])
        seconds.append(divmod(s,60.0)[1])
        
    time_info.append(seconds)
    hours_temp = []
    minutes = []
    for m in minutes_temp : 
        hours_temp.append(divmod(m,60.0)[0])
        minutes.append(divmod(m,60.0)[1])
        
    time_info.append(minutes)
    days = []
    hours = []
    for h in hours_temp : 
        days.append(divmod(h,60.0)[0])
        hours.append(divmod(h,60.0)[1])
    
    time_info.append(hours)
    time_info.append(days)
     
    return time_info

In [37]:
get_time(timestamps)[0]

[0.0,
 0.0,
 41.708333333333336,
 83.41666666666667,
 125.12499999999999,
 166.83333333333334,
 208.54166666666666,
 250.24999999999997,
 291.9583333333333,
 333.6666666666667,
 375.375,
 417.0833333333333,
 458.79166666666663,
 500.49999999999994,
 542.2083333333333,
 583.9166666666666,
 625.625,
 667.3333333333334,
 709.0416666666666,
 750.75,
 792.4583333333333,
 834.1666666666666,
 875.875,
 917.5833333333333,
 959.2916666666666,
 0.9999999999998863,
 42.70833333333326,
 84.41666666666652,
 126.125,
 167.83333333333326,
 209.54166666666674,
 251.25,
 292.95833333333326,
 334.66666666666674,
 376.375,
 418.08333333333326,
 459.7916666666665,
 501.5,
 543.2083333333333,
 584.9166666666665,
 626.625,
 668.3333333333333,
 710.0416666666667,
 751.75,
 793.4583333333335,
 835.1666666666665,
 876.8749999999998,
 918.5833333333333,
 960.2916666666665,
 1.9999999999997726,
 43.708333333333485,
 85.41666666666652,
 127.125,
 168.83333333333303,
 210.54166666666652,
 252.25,
 293.958333333333

In [ ]:
'float' object is not iterable

In [16]:
detect_video = 'test1.mp4'
videoCapture = cv2.VideoCapture(detect_video)
success, frame = videoCapture.read()
frame_counter = 1
frame_total = count_frames(detect_video)

# Facenet

In [1]:
import numpy as np
import os, time

import cv2

from skimage.transform import resize

from scipy.spatial import distance

from keras.models import load_model

Using TensorFlow backend.


In [4]:
os.getcwd()

'D:\\Taishin'

In [2]:
#驗證 compares 列表中相片的人與 valid 相片中的人

valid = "1.png"

compares = ['2.png', '80.png', '250.png' ]

#用 OpenCV 的 Cascade classifier 來偵測臉部，不一定跟 Facenet 一樣要用 MTCNN。
newpath = 'D:\\Anaconda\\Library\\etc\\haarcascades\\'
cascade_path = newpath+"haarcascade_frontalface_default.xml"

#我們的人像相片都放置於 validPicPath

validPicPath = 'face3_160\\'

#此版 Facenet model 需要的相片尺寸為 160×160

image_size = 160

#使用 MS-Celeb-1M dataset pretrained 好的 Keras model

model_path ="model\\facenet_keras.h5"

model = load_model(model_path)

#————————————————————

#圖像白化（whitening）可用於對過度曝光或低曝光的圖片進行處理，處理的方式就是改變圖像的平均像素值為 0 ，改變圖像的方差為單位方差 1。

def prewhiten(x):

    if x.ndim == 4:

        axis = (1, 2, 3)

        size = x[0].size

    elif x.ndim == 3:

        axis = (0, 1, 2)

        size = x.size

    else:

        raise ValueError("Dimension should be 3 or 4")

    mean = np.mean(x, axis=axis, keepdims=True)

    std = np.std(x, axis=axis, keepdims=True)

    std_adj = np.maximum(std, 1.0/np.sqrt(size))

    y = (x-mean) / std_adj

    return y

#使用 L1 或 L2 標準化圖像，可強化其特徵。

def l2_normalize(x, axis=-1, epsilon=1e-10):

    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))

    return output

#偵測並取得臉孔 area，接著再 resize 為模型要求的尺寸（下方例子並未作alignment）

def align_image(img, margin):

    cascade = cv2.CascadeClassifier(cascade_path)

    faces = cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3)

    if(len(faces)>0):

        (x, y, w, h) = faces[0]

        face = img[y:y+h, x:x+w]

        faceMargin = np.zeros((h+margin*2, w+margin*2, 3), dtype = "uint8")

        faceMargin[margin:margin+h, margin:margin+w] = face

        cv2.imwrite(str(time.time())+".png",img)

        aligned = resize(faceMargin, (image_size, image_size), mode="reflect")

        cv2.imwrite(str(time.time())+"_aligned.png",aligned)

        return aligned

    else:

        return None

#圖像的預處理（即前述的幾項步驟）

def preProcess(img):

    whitenImg = prewhiten(img)

    whitenImg = whitenImg[np.newaxis, :]

    return whitenImg

#————————————————-

imgValid = validPicPath + valid

aligned = align_image(cv2.imread(imgValid), 6)

if(aligned is None):

    print("Cannot find any face in image: {}".format(imgValid))

else:

    faceImg = preProcess(aligned)

#–> model 會輸出 128 維度的臉孔特徵向量，接著我們將它們合併並進行 L2 正規化。Z

embs_valid = l2_normalize(np.concatenate(model.predict(faceImg)))

#同上方的 valid 圖片，依序取得各圖片人臉的臉孔特徵向量，再與 valid 進行歐氏距離計算。

for member in compares:

    img_file = validPicPath + member

    aligned = align_image(cv2.imread(img_file), 6)

    if(aligned is not None):

        faceImg = preProcess(aligned)

        embs = l2_normalize(np.concatenate(model.predict(faceImg)))

        distanceNum = distance.euclidean(embs_valid, embs)

        print("Diff with {} is {}".format(member, distanceNum))













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


D:\Anaconda\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Diff with 80.png is 1.2336183786392212
Diff with 250.png is 0.48661112785339355


# Webcam + dlib

In [1]:
from imutils.face_utils import FaceAligner
from imutils.video import count_frames
import imutils
import numpy as np
import dlib
# import os
# import glob
# from skimage.measure import compare_ssim
import time
import cv2

def load_caffe_models():
    path = 'D:\\Taishin\\'
    age_net = cv2.dnn.readNetFromCaffe(path+'deploy_age.prototxt', 'age_net.caffemodel')

    gender_net = cv2.dnn.readNetFromCaffe(path+'deploy_gender.prototxt', 'gender_net.caffemodel')

    return(age_net, gender_net)

# def detect(frame):
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = detector(gray, 1)
#     for face in faces:
#         faceAligned = fa.align(gray, gray, face)
#     return(faceAligned)

In [ ]:
# Dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# fa = FaceAligner(predictor, desiredFaceWidth=160)
# DNN
age_net, gender_net = load_caffe_models() # 預測模型讀取(年齡、性別)

video_capture = cv2.VideoCapture(0)
video_capture.set(3, 480) #set width of the frame
video_capture.set(4, 640) #set height of the frame 

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']

gender_list = ['Male', 'Female']

while True :
    ret, frame = video_capture.read()
    frame = cv2.flip(frame,1) # 影片鏡面呈現
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)
    
#     for face in faces:
#         faceAligned = fa.align(gray, gray, face)
    
#     if(len(faces)>0):
#         print("Found {} faces".format(str(len(faces))))
    for face in faces :
#         print(face.left())
        x = face.left()
        y = face.top()
        w = face.right() - face.left()
        h = face.bottom() - face.top()
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2) # (0, 255, 0), 2)
        
#Get Face 
    face_img = frame[y:y+h, h:h+w].copy()
    blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

#Predict Gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]
    print("Gender : " + gender)

#Predict Age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_list[age_preds[0].argmax()]
    print("Age Range: " + age)

    overlay_text = "%s %s" % (gender, age)
    cv2.putText(frame, overlay_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255) , 2, cv2.LINE_AA)

    cv2.imshow('frame', frame)  
#0xFF is a hexadecimal constant which is 11111111 in binary.
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
video_capture.release()
cv2.destroyAllWindows()

Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Gender : Male
Age Range: (25, 32)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6

Gender : Female
Age Range: (25, 32)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (25, 32)
Gender : Male
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Male
Age Range: (25, 32)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (25, 32)
Gender : Male
Age Range: (38, 43)
Gender : Male
Age Range: (38, 43)
Gender : Male
Ag

Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gende

Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (48, 53)
Gender : Female
Age Range: (48, 53)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (0, 2)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (0, 2)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (0, 2)
Gender : Male
Age Range: (48, 53)
Gender : Female
Age Range: (48, 53)
Gender : Male
Age Range: (0, 2)
Gend

Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (

...

Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age 

Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (60, 100)
Gender : Male
Age Range: (0, 2)
Gender : Male
Age Range: (60, 100)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (38, 43)
Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (15, 20)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Gender : Female
Age Range: (8, 12)
Gender : Male
Age Range: (25, 32)
Gender : Male
Age Range: (25, 32)
Gender : Male
Age Rang

Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (0, 2)
Gender : Female
Age Range: (0, 2)
Gender : Female
Age Range: (0, 2)
Gender : Female
Age Range: (0, 2)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (0, 2)
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (48, 53)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (15, 20)
Gen

Gender : Male
Age Range: (48, 53)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (8, 12)
Gender : Male
Age Range: (15, 20)
Gender : Male
Age Range: (8, 12)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (8, 12)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (0, 2)
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Female
Age Range: (25, 32)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (4, 6)
Gender :

Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (15, 20)
Gender : Female
Age Range: (15, 20)
Gender : Male
Age Range: (4, 6)
Gender : Male
Age Range: (0, 2)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (4, 6)
Gender : Female
Age Range: (15, 20)
Gender : Female


In [ ]:
video_capture.release()
cv2.destroyAllWindows()

# Gender, Age , Emotion 

In [2]:
from imutils.face_utils import FaceAligner
from imutils.video import count_frames
import imutils
import numpy as np
import dlib
# import os
# import glob
# from skimage.measure import compare_ssim
import time
import cv2

def load_caffe_models():
    path = 'D:\\Taishin\\'
    age_net = cv2.dnn.readNetFromCaffe(path+'deploy_age.prototxt', 'age_net.caffemodel')

    gender_net = cv2.dnn.readNetFromCaffe(path+'deploy_gender.prototxt', 'gender_net.caffemodel')

    return(age_net, gender_net)

# def detect(frame):
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = detector(gray, 1)
#     for face in faces:
#         faceAligned = fa.align(gray, gray, face)
#     return(faceAligned)

In [3]:
path = 'D:\\Taishin\\'
emotion_net = cv2.dnn.readNetFromCaffe(path+'deploy.prototxt', 'EmotiW_VGG_S.caffemodel.caffemodel')

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1132: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "EmotiW_VGG_S.caffemodel.caffemodel" in function 'cv::dnn::ReadProtoFromBinaryFile'


In [1]:
# Dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# fa = FaceAligner(predictor, desiredFaceWidth=160)
# DNN
age_net, gender_net = load_caffe_models() # 預測模型讀取(年齡、性別)

video_capture = cv2.VideoCapture(0)
video_capture.set(3, 480) #set width of the frame
video_capture.set(4, 640) #set height of the frame 

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']

gender_list = ['Male', 'Female']

emotion_net = load_model("D:/Taishin/model_v6_23.hdf5")

while True :
    ret, frame = video_capture.read()
    frame = cv2.flip(frame,1) # 影片鏡面呈現
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)
    
#     for face in faces:
#         faceAligned = fa.align(gray, gray, face)
    
#     if(len(faces)>0):
#         print("Found {} faces".format(str(len(faces))))
    for face in faces :
#         print(face.left())
        x = face.left()
        y = face.top()
        w = face.right() - face.left()
        h = face.bottom() - face.top()
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2) # (0, 255, 0), 2)
        
# Get Face 
    face_img = frame[y:y+h, h:h+w].copy()
    blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

# Predict Gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]
    print("Gender : " + gender)

# Predict Age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_list[age_preds[0].argmax()]
    print("Age Range: " + age)
    
# Predict Emotion
    
#     emotion_net.predict(blob)
    


    overlay_text = "%s %s" % (gender, age)
    cv2.putText(frame, overlay_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255) , 2, cv2.LINE_AA)

    cv2.imshow('frame', frame)  
#0xFF is a hexadecimal constant which is 11111111 in binary.
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
video_capture.release()
cv2.destroyAllWindows()

NameError: name 'dlib' is not defined

In [12]:
video_capture.release()

In [6]:
RD = cv2.imread('D:/Taishin/RD.png')

In [7]:
RD_gray = cv2.cvtColor(RD, cv2.COLOR_RGB2GRAY)

In [8]:
cv2.imwrite('D:/Taishin/RD_gray.jpg',RD_gray)

True